In [3]:
from scripts.tools.issuetracker import *

In [4]:
import os

api = IssuetrackerAPI("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist')
os.makedirs("pdfs4", exist_ok=True)
gantt = api.download_gantt('pbssoftware')
gantt = list(gantt)



In [56]:
pbscs_path = "C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy"
pdfs_path = "pdfs4"
index = "IS00038B1 Rev A Hello v3 FRS Update.pdf"

In [58]:
import shutil

try:
    shutil.rmtree(pdfs_path)
except FileNotFoundError:
    pass
os.makedirs(pdfs_path, exist_ok=True)
shutil.copy2(pbscs_path + "/" + index, pdfs_path + "/" + index)

def fn_fix(fn):
    return fn.replace("'", "").replace('"', "").replace("/", "")

files = []
for i, iss in enumerate(gantt, 1):
    ffrom = "pdfs3/%d.pdf" % iss.id
    fto = pdfs_path + "/%d Issue #%d- %s.pdf" % (i, iss.id, fn_fix(iss.subject))
    if os.path.exists(ffrom):
        print("Copying %s to %-10s..." % (ffrom, fto))
        shutil.copy2(ffrom, fto)
        files.append((fto, iss))

Copying pdfs3/420.pdf to pdfs4/5 Issue #420- Database sometimes gets corrupt.pdf...
Copying pdfs3/989.pdf to pdfs4/6 Issue #989- Login Behavior.pdf...
Copying pdfs3/1073.pdf to pdfs4/8 Issue #1073- Changes to Existing User Events.pdf...
Copying pdfs3/2115.pdf to pdfs4/9 Issue #2115- Web UI Login events show Unknown User.pdf...
Copying pdfs3/2445.pdf to pdfs4/10 Issue #2445- Web UI Logout unknown user.pdf...
Copying pdfs3/2458.pdf to pdfs4/11 Issue #2458- End Batch should be last user event recorded for batch.pdf...
Copying pdfs3/2906.pdf to pdfs4/12 Issue #2906- Changes to Load Bag behavior.pdf...
Copying pdfs3/2908.pdf to pdfs4/13 Issue #2908- Changes to Clearing Alarms behavior.pdf...
Copying pdfs3/1262.pdf to pdfs4/15 Issue #1262- Recipe Steps in Reports are out of order.pdf...
Copying pdfs3/1263.pdf to pdfs4/16 Issue #1263- Errors report csv has entries in wrong columns.pdf...
Copying pdfs3/1759.pdf to pdfs4/17 Issue #1759- Auto archive DB and create new one.pdf...
Copying pdfs3/18

In [62]:
def quotify(f):
    if " " in f:
        return '"%s"' % f
    
import subprocess

def find_pdf_info(fp):
    cmd = "pdftk %s dump_data" % quotify(fp)
    code, data = subprocess.getstatusoutput(cmd)
    if code != 0:
        raise ValueError(data)
    data = data.splitlines()
    it = iter(data)
    npgs = None
    title = None
    while True:
        line = next(it, None)
        if line is None:
            break
        if line == "InfoBegin":
            key = next(it, None)
            val = next(it, None)
            if key is None or val is None:
                raise ValueError("Got corrupt dump file for %s" % fp)
            if key.split(": ")[1] == 'Title':
                title = val.split(": ")[1]
        else:
            try:
                k,v = line.split(": ")
            except ValueError:
                continue
            else:
                if k == 'NumberOfPages':
                    npgs = int(v)
            
    if npgs is None:
        raise ValueError("Failed to find number of pages")
    return title, npgs
    
    

In [64]:
def get_title(issue):
    return "#%d: %s" % (issue.id, issue.subject)

def add_file(inputs, srclines, f, s):
    srclines.append(s)
    inputs.append('"%s"' % f)

In [99]:
ps_tmplt = "[/Page %d /View [/XYZ null null null] /Title (%s) /OUT pdfmark"
pdf_index = pdfs_path + "/" + index
_, npgs = find_pdf_info(pdf_index)
srclines = []
inputs = []
i = 1
add_file(inputs, srclines, pdf_index, ps_tmplt % (i, "Issues Index"))
i += npgs

np = 1
#fused = files[:np+1]
fused = files
last = 0
for n, (f, iss) in enumerate(fused, 1):
    print("\rFinding info for '%-20s' (%d/%d)%s" % (f, n, len(fused), " "*max(last-20, 0)), end="")
    last = len(f)
    title, npgs = find_pdf_info(f)
    s = ps_tmplt % (i, get_title(iss))
        
    add_file(inputs, srclines, f, s)
        
    i += npgs
    
print()


Finding info for 'pdfs4/161 Issue #3012- Centralize Data recording.pdf' (92/92)                                                              


In [100]:
src = " ".join(srclines)
mk_toc = "mk_toc.ps"
cmd = "gswin64c.exe -dBATCH -dNOPAUSE -sDEVICE=pdfwrite -sOutputFile=Updated.pdf %s -f " % mk_toc + " ".join(inputs)

def run_cmd(cmd, src, mk_toc):
    
    with open(mk_toc, 'w') as f:
        f.write("\n".join(srclines))
    try:
        return subprocess.getoutput(cmd)
    finally:
        try:
            os.remove(mk_toc)
        except OSError:
            pass
        

In [101]:
r = run_cmd(cmd, src, mk_toc)
print(r)

GPL Ghostscript 9.19 (2016-03-23)
Copyright (C) 2016 Artifex Software, Inc.  All rights reserved.
This software comes with NO WARRANTY: see the file PUBLIC for details.
Processing pages 1 through 3.
Page 1
Page 2
Page 3
**** Warning: Discovered more enties in xref than declared in trailer /Size
Processing pages 1 through 4.
Page 1
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
Page 2
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
Page 3
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
Page 4
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
   **** Warning: Ignoring bad ToUnicode CMap.
Loading NimbusSanL-Reg font from %rom%Resource/Font/NimbusSanL-Reg... 5124264 3211551 14420552

gswin64c.exe -dBATCH -dNOPAUSE -sDEVICE=pdfwrite -sOutputFile=Updated.pdf %s -f "pdfs4/IS00038B1 Rev A Hello v3 FRS Update.pdf" "pdfs4/5 Issue #420- Database sometimes gets corrupt.pdf" "pdfs4/6 Issue #989- Login Behavior.pdf" "pdfs4/8 Issue #1073- Changes to Existing User Events.pdf" "pdfs4/9 Issue #2115- Web UI Login events show Unknown User.pdf" "pdfs4/10 Issue #2445- Web UI Logout unknown user.pdf" "pdfs4/11 Issue #2458- End Batch should be last user event recorded for batch.pdf" "pdfs4/12 Issue #2906- Changes to Load Bag behavior.pdf" "pdfs4/13 Issue #2908- Changes to Clearing Alarms behavior.pdf" "pdfs4/15 Issue #1262- Recipe Steps in Reports are out of order.pdf" "pdfs4/16 Issue #1263- Errors report csv has entries in wrong columns.pdf" "pdfs4/17 Issue #1759- Auto archive DB and create new one.pdf" "pdfs4/19 Issue #1845- Make sure RIO time is correct.pdf" "pdfs4/20 Issue #1885- Webcontrol.conf file .pdf" "pdfs4/22 Issue #2309- Opening Reports in WebUI in Kiosk Mode (Christian pl